In [10]:
from gpt4all import GPT4All
import pandas as pd
import requests
import json
import os
from datasets import load_dataset
import time

In [11]:
model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")


In [12]:
dataset = load_dataset("PatronusAI/financebench")


In [13]:
df = pd.DataFrame(dataset['train'])
df.head()

,financebench_id,doc_name,doc_link,doc_period,question_type,question,answer,evidence_text,page_number
0,financebench_id_03029,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,What is the FY2018 capital expenditure amount ...,$1577.00,Table of Contents \n3M Company and Subsidiarie...,60
1,financebench_id_04672,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,Assume that you are a public equities analyst....,$8.70,Table of Contents \n3M Company and Subsidiarie...,58
2,financebench_id_00499,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,Is 3M a capital-intensive business based on FY...,"No, the company is managing its CAPEX and Fixe...",3M Company and Subsidiaries\n Consolidated Sta...,"48,50,52"
3,financebench_id_01226,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,What drove operating margin change as of FY202...,Operating Margin for 3M in FY2022 has decrease...,"SG&A, measured as a percent of sales, increase...",27
4,financebench_id_01865,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,novel-generated,"If we exclude the impact of M&A, which segment...",The consumer segment shrunk by 0.9% organically.,Worldwide Sales Change\nBy Business Segment Or...,25


In [14]:
examples = [ f"<answer> {row['answer']} </answer>" for idx, row in df.iloc[:50].iterrows() ]
examples

['<answer> $1577.00 </answer>',
 '<answer> $8.70 </answer>',
 '<answer> No, the company is managing its CAPEX and Fixed Assets pretty efficiently, which is evident from below key metrics:\nCAPEX/Revenue Ratio: 5.1%\nFixed assets/Total Assets: 20%\nReturn on Assets= 12.4% </answer>',
 '<answer> Operating Margin for 3M in FY2022 has decreased by 1.7% primarily due to: \n-Decrease in gross Margin\n-mostly one-off charges including Combat Arms Earplugs litigation, impairment related to exiting PFAS manufacturing, costs related to exiting Russia and divestiture-related restructuring\ncharges </answer>',
 '<answer> The consumer segment shrunk by 0.9% organically. </answer>',
 "<answer> No. The quick ratio for 3M was 0.96 by Jun'23 close, which needs a bit of an improvement to touch the 1x mark </answer>",
 "<answer> Following debt securities registered under 3M's name are listed to trade on the New York Stock Exchange:\n-1.500% Notes due 2026 (Trading Symbol: MMM26)\n-1.750% Notes due 2030 (

In [15]:
# Placeholder prompts
HUMAN_PROMPT = "Hello! I'm seeking information about financial reports for a research project."
AI_PROMPT = " Please respond with concise, factual information based on the provided context and examples."

# List to store synthesized responses
syntheses_with_10_labels = []

# Loop through the DataFrame rows
start_time = time.time()
for idx, row in df.iterrows():
    question = row['question']
    evidence_text = row['evidence_text']

    # Define structured conversation template
    system_template = f"You are an accurate chatbot specialized in responding to queries about 10-K documents. Please only respond in the manner demonstrated by the following examples. If an explanation wasn't asked, refrain from providing one. "
    f"Based on the following examples: {examples[:10]}, please provide your answer in the same format. "
    f"please address the question: {question}. "
    f"Consider the provided text as evidence: {evidence_text}. "
    
    prompt_template = (
        f"USER: {question}\n"
        f"ASSISTANT: "
    )
    
    #initiate chat 
    with model.chat_session(system_template, prompt_template):
        response = model.generate(prompt=AI_PROMPT, temp=0.7)
        print(response)
        print(idx)      
        print("idx")
        end_time = time.time()  # End the timer
        print(f"Time taken for this idx: {end_time - start_time} seconds")
        syntheses_with_10_labels.append(response)

3M reported a total capital expenditure of $6,945 million in their fiscal year 2018 cash flow statement.
0
idx
Time taken for this idx: 60.2633330821991 seconds
3M Company's fiscal year ends on December 31st, which means their financial statements are as of and for the period from January 1st to December 31st. According to the company's balance sheet as of June 2018, the net profit for the period from January 1st to June 30th was $7.5 billion USD. Therefore, the year end FY2018 net PPNE for 3M is approximately $7.5 billion USD.
1
idx
Time taken for this idx: 142.03822588920593 seconds
3M is a capital-intensive business based on its financial statements for fiscal year 2022. The company invested heavily in research and development, which led to an increase in net debt.
2
idx
Time taken for this idx: 181.87700510025024 seconds
3M's operating margin change as of FY2022 can be attributed to various factors such as changes in sales volume, pricing strategy, cost structure, and product mix. 

In [16]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_10_labels}).to_csv('syntheses_10.csv',index=False)


In [17]:
# Placeholder prompts
HUMAN_PROMPT = "Hello! I'm seeking information about financial reports for a research project."
AI_PROMPT = " Please respond with concise, factual information based on the provided context and examples."

# List to store synthesized responses
syntheses_with_20_labels = []

# Loop through the DataFrame rows
start_time = time.time()
for idx, row in df.iterrows():
    question = row['question']
    evidence_text = row['evidence_text']

    # Define structured conversation template
    system_template = f"You are an accurate chatbot specialized in responding to queries about 10-K documents. Please only respond in the manner demonstrated by the following examples. If an explanation wasn't asked, refrain from providing one. "
    f"Based on the following examples: {examples[:20]}, please provide your answer in the same format. "
    f"please address the question: {question}. "
    f"Consider the provided text as evidence: {evidence_text}. "
    
    prompt_template = (
        f"USER: {question}\n"
        f"ASSISTANT: "
    )
    
    #initiate chat 
    with model.chat_session(system_template, prompt_template):
        response = model.generate(prompt=AI_PROMPT, temp=0.7)
        print(response)
        print(idx)      
        print("idx")
        end_time = time.time()  # End the timer
        print(f"Time taken for this idx: {end_time - start_time} seconds")
        syntheses_with_20_labels.append(response)

3M's fiscal year ends December 31st, and their cash flow statement shows that their capital expenditures totaled $20.5 billion for FY2018. Therefore, the answer to your question is $20.5 million.
0
idx
Time taken for this idx: 26.506649017333984 seconds
3M Company's fiscal year ends on December 31st, which means their financial statements are as of and for the period from January 1st to December 31st. As shown in their balance sheet dated March 29th, 2018, the company had total assets of $47.5 billion USD at year end FY2018. Therefore, the net profit for the period from January 1st to December 31st, which is also known as the fiscal quarter, was approximately $6.9 billion USD.
1
idx
Time taken for this idx: 66.9532630443573 seconds
3M is a capital-intensive business based on its financial statements for fiscal year 2022. To answer your question, I would need access to the full report or summary of financial information for 3M's fiscal year 2022. Please provide me with that information 

In [18]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_20_labels}).to_csv('syntheses_20.csv',index=False)


In [25]:
# Placeholder prompts
HUMAN_PROMPT = "Hello! I'm seeking information about financial reports for a research project."
AI_PROMPT = " Please respond with concise, factual information based on the provided context and examples."

# List to store synthesized responses
syntheses_with_30_labels = []

# Loop through the DataFrame rows
start_time = time.time()
for idx, row in df.iterrows():
    question = row['question']
    evidence_text = row['evidence_text']

    # Define structured conversation template
    system_template = f"You are an accurate chatbot specialized in responding to queries about 10-K documents. Please only respond in the manner demonstrated by the following examples. If an explanation wasn't asked, refrain from providing one. "
    f"Based on the following examples: {examples[:30]}, please provide your answer in the same format. "
    f"please address the question: {question}. "
    f"Consider the provided text as evidence: {evidence_text}. "
    
    prompt_template = (
        f"USER: {question}\n"
        f"ASSISTANT: "
    )
    
    #initiate chat 
    with model.chat_session(system_template, prompt_template):
        response = model.generate(prompt=AI_PROMPT, temp=0.7)
        print(response)
        print(idx)      
        print("idx")
        end_time = time.time()  # End the timer
        print(f"Time taken for this idx: {end_time - start_time} seconds")
        syntheses_with_30_labels.append(response)

3M reported total capital expenditures of $6,945 million in their fiscal year 2018 cash flow statement. This information can be found on page 72 of the report.
0
idx
Time taken for this idx: 36.77517890930176 seconds
3M Company's fiscal year ends on December 31st, which means their financial statements are as of and for the period from January 1st to December 31st. As shown in their balance sheet dated March 29th, 2018, 3M reported net profit after tax (PPNT) of $7.4 billion for fiscal year 2018. Therefore, the year end FY2018 net PPNE for 3M is $7.4 billion.
1
idx
Time taken for this idx: 79.21921300888062 seconds
3M is a capital-intensive business based on its financial statements for fiscal year 2022. The company's investments in property, plant, and equipment totaled $8.4 billion during the year, which reflects the high level of investment required to maintain and grow their product offerings and manufacturing capabilities.
2
idx
Time taken for this idx: 106.31389498710632 seconds


In [27]:
# Placeholder prompts
HUMAN_PROMPT = "Hello! I'm seeking information about financial reports for a research project."
AI_PROMPT = " Please respond with concise, factual information based on the provided context and examples."

# List to store synthesized responses
syntheses_with_40_labels = []

# Loop through the DataFrame rows
start_time = time.time()
for idx, row in df.iterrows():
    question = row['question']
    evidence_text = row['evidence_text']

    # Define structured conversation template
    system_template = f"You are an accurate chatbot specialized in responding to queries about 10-K documents. Please only respond in the manner demonstrated by the following examples. If an explanation wasn't asked, refrain from providing one. "
    f"Based on the following examples: {examples[:40]}, please provide your answer in the same format. "
    f"please address the question: {question}. "
    f"Consider the provided text as evidence: {evidence_text}. "
    
    prompt_template = (
        f"USER: {question}\n"
        f"ASSISTANT: "
    )
    
    #initiate chat 
    with model.chat_session(system_template, prompt_template):
        response = model.generate(prompt=AI_PROMPT, temp=0.7)
        print(response)
        print(idx)      
        print("idx")
        end_time = time.time()  # End the timer
        print(f"Time taken for this idx: {end_time - start_time} seconds")
        syntheses_with_40_labels.append(response)

3M's capital expenditure amount for fiscal year 2018 was $4,675 million.
0
idx
Time taken for this idx: 21.629245042800903 seconds
3M Company's fiscal year ends on December 31st, therefore its net profit after tax (PPNT) can be calculated by subtracting total expenses from total revenue. The company reported a net profit of $6 billion in their FY2018 10-K document. Therefore, the year end FY2018 net PPNE for 3M is $6 billion.
1
idx
Time taken for this idx: 56.74147272109985 seconds
3M is a capital-intensive business based on its financial statements for fiscal year 2022. The company's cash flow statement shows significant outflows related to investing activities, including purchases of property, plant, and equipment (PPE) and intangible assets. Additionally, the company's debt-to-equity ratio is high, indicating that it has a large amount of debt relative to its equity. These factors suggest that 3M invests heavily in research and development to create new products and improve existing

In [26]:
# Placeholder prompts
HUMAN_PROMPT = "Hello! I'm seeking information about financial reports for a research project."
AI_PROMPT = " Please respond with concise, factual information based on the provided context and examples."

# List to store synthesized responses
syntheses_with_50_labels = []

# Loop through the DataFrame rows
start_time = time.time()
for idx, row in df.iterrows():
    question = row['question']
    evidence_text = row['evidence_text']

    # Define structured conversation template
    system_template = f"You are an accurate chatbot specialized in responding to queries about 10-K documents. Please only respond in the manner demonstrated by the following examples. If an explanation wasn't asked, refrain from providing one. "
    f"Based on the following examples: {examples[:50]}, please provide your answer in the same format. "
    f"please address the question: {question}. "
    f"Consider the provided text as evidence: {evidence_text}. "
    
    prompt_template = (
        f"USER: {question}\n"
        f"ASSISTANT: "
    )
    
    #initiate chat 
    with model.chat_session(system_template, prompt_template):
        response = model.generate(prompt=AI_PROMPT, temp=0.7)
        print(response)
        print(idx)      
        print("idx")
        end_time = time.time()  # End the timer
        print(f"Time taken for this idx: {end_time - start_time} seconds")
        syntheses_with_50_labels.append(response)

3M's capital expenditure amount for fiscal year 2018 was $5,694 million. This information is provided in the company's cash flow statement, which can be found on page 7 of their annual report.
0
idx
Time taken for this idx: 25.825327157974243 seconds
3M Company's Year End Fiscal Year (FY) 2018 Net Profit from Operations (PPNO), which is calculated by subtracting total operating expenses from total revenues, was approximately USD 45 billion.
1
idx
Time taken for this idx: 53.46230912208557 seconds
3M is a capital-intensive business based on its financial statements for fiscal year 2022. The company invested heavily in research and development, purchasing equipment, and expanding facilities during the period. Additionally, the company's investments in employee training and development contributed to higher labor costs.
2
idx
Time taken for this idx: 77.46065616607666 seconds
3M reported an operating margin of 18.7% in fiscal year (FY) 2022. This was driven by several factors including in

In [28]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_30_labels}).to_csv('syntheses_30.csv',index=False)


In [29]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_40_labels}).to_csv('syntheses_40.csv',index=False)


In [30]:
pd.DataFrame({'question': df['question'], 'answer' : df['answer'], 'evidence_text' : df['evidence_text'], 'syntheses' : syntheses_with_50_labels}).to_csv('syntheses_50.csv',index=False)
